In [ ]:
import json, utils, hashlib, random, re
from ecdsa import SigningKey, SECP256k1

In [ ]:
# Essential part
vout_amount = [5000, 1000] # tx vout amount; vout_cnt 길이의 리스트

vout_idx = 2 # UTXO vout
utxo_amount = 8000 # UTXO amount

# P2PKH -> 2-1

# M of N MULTISIGNATURE -> 2-2
m=3
n=5

# script for P2SH -> 2-3-1
# <pubKey> : for CHECKSIG
# <pubKeyHash> : for P2PKH
# <MultiPubKey> : for MULTISIGNATURE
redeem_script = "IF <pubKey> CHECKSIG ELSE <pubKey> CHECKSIG ENDIF"

# Ex1) IF 2 <MultiPubKey> <MultiPubKey> <MultiPubKey> 3 CHECKMULTISIG ELSE DUP HASH <pubKeyHash> EQUALVERIFY CHECKSIG ENDIF
# Ex2) IF <pubKeyHash> CHECKSIGVERIFY <pubKeyHash> CHECKSIG ENDIF
# Ex3) <pubKey> CHECKSIG

In [92]:
# 1
tx={
        "txid": "",
        "vin":[
            {
                "ptxid":"",
                "vout": 0,
                "scriptSig": ""
            }
        ],
        "vout": [
        ]
    }

# create vout
sum = 0
for i, amount in enumerate(vout_amount):
    o_private_key = SigningKey.generate(curve=SECP256k1)
    o_public_key = o_private_key.get_verifying_key().to_string().hex()

    o_pubKey_hash = utils.sha256_ripemd160(o_public_key)
    o_scriptPubKey = " ".join(["DUP", "HASH", o_pubKey_hash, "EQUALVERIFY", "CHECKSIG"])
    tx["vout"].append({"amount":amount, "scriptPubKey":o_scriptPubKey})
    sum += amount

# refund utxo
if sum < utxo_amount:
    o_private_key = SigningKey.generate(curve=SECP256k1)
    o_public_key = o_private_key.get_verifying_key().to_string().hex()

    o_pubKey_hash = utils.sha256_ripemd160(o_public_key)
    o_scriptPubKey = " ".join(["DUP", "HASH", o_pubKey_hash, "EQUALVERIFY", "CHECKSIG"])
    tx["vout"].append({"amount":utxo_amount-sum, "scriptPubKey":o_scriptPubKey})

# create new UTXO
ptxid = SigningKey.generate(curve=SECP256k1).to_string().hex()
ptxid = hashlib.sha256(ptxid.encode()).hexdigest()

UTXO = {
            "ptxid": ptxid,
            "vout": vout_idx,
            "amount": utxo_amount,
            "scriptPubKey": ""
        }

tx["vin"][0]["ptxid"] = ptxid
tx["vin"][0]["vout"] = vout_idx

In [ ]:
# 2-1.
# P2PKH generate
private_key = SigningKey.generate(curve=SECP256k1)
public_key = private_key.get_verifying_key().to_string().hex()

# P2PKH scriptPubKey
pubKey_hash = utils.sha256_ripemd160(public_key)
scriptPubKey = " ".join(["DUP", "HASH", pubKey_hash, "EQUALVERIFY", "CHECKSIG"])

UTXO["scriptPubKey"] = scriptPubKey
verifying_tx = utils.sha256_twice(tx["vin"], tx["vout"], UTXO["scriptPubKey"])
signature = private_key.sign(verifying_tx)
scriptSig = " ".join([signature.hex(), public_key])

# P2PKH verification
print(f"Validate check: {utils.sig_validation_check(public_key, signature.hex(), verifying_tx)}")

In [ ]:
# 2-2.
# MULTISIGNATURE generator
scriptSig = []
scriptPubKey = []
signature = []
scriptPubKey.append(str(m))

for _ in range(n):
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    scriptSig.append(private_key)
    scriptPubKey.append(public_key.to_string().hex())
    
scriptPubKey.append(str(n))
scriptPubKey.append("CHECKMULTISIG")
scriptSig = random.sample(scriptSig, m)

# make signature to tx
UTXO["scriptPubKey"] = " ".join(scriptPubKey)
verifying_tx = utils.sha256_twice(tx["vin"], tx["vout"], UTXO["scriptPubKey"])

for i in range(m):
    scriptSig[i] = scriptSig[i].sign(verifying_tx).hex()
    signature.append(scriptSig[i])

scriptSig = " ".join(signature)

# MULTISIG verification
pubKeys = scriptPubKey[1:1+n]
val = 0
for pubKey in pubKeys:
    for sig in signature:
        if utils.sig_validation_check(pubKey, sig, verifying_tx):
            val += 1

print(f"val = {val}, m = {m}")

In [ ]:
# 2-3-1.
# P2SH generator
origin = redeem_script
pubKey_cnt = 0
MULTI_cnt = 0
P2PKH_cnt = 0
MULTIsig_number = []
MULTI_pattern = r"<MultiPubKey>"
P2PKH_pattern = r"<pubKeyHash>"
pubKey_pattern = r"<pubKey>"
MULTIsig_pattern = r"\b\d+\b"
for token in redeem_script.split():
    if re.match(pubKey_pattern, token):
        pubKey_cnt += 1
    elif re.match(MULTI_pattern, token):
        MULTI_cnt += 1
    elif re.match(P2PKH_pattern, token):
        P2PKH_cnt += 1
    elif re.match(MULTIsig_pattern, token):
        MULTIsig_number.append(int(token))

pubKey_privKeys = []
pubKey_pubKeys = []
for _ in range(pubKey_cnt):
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    pubKey_privKeys.append(private_key)
    pubKey_pubKeys.append(public_key.to_string().hex())

MULTI_privKeys = []
MULTI_pubKeys = []
for _ in range(MULTI_cnt):
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    MULTI_privKeys.append(private_key)
    MULTI_pubKeys.append(public_key.to_string().hex())

P2PKH_privKeys = []
P2PKH_pubKeys = []
for _ in range(P2PKH_cnt):
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    P2PKH_privKeys.append(private_key)
    P2PKH_pubKeys.append(public_key.to_string().hex())

for pubKey in pubKey_pubKeys:
    redeem_script = re.sub(pubKey_pattern, pubKey, redeem_script, count=1)
    
for pubKey in MULTI_pubKeys:
    redeem_script = re.sub(MULTI_pattern, pubKey, redeem_script, count=1)

for pubkeyhash in P2PKH_pubKeys:
    redeem_script = re.sub(P2PKH_pattern, utils.sha256_ripemd160(pubkeyhash), redeem_script, count=1)

scriptPubKey = " ".join(["DUP", "HASH", utils.sha256_ripemd160(redeem_script), "EQUALVERIFY"])
UTXO["scriptPubKey"] = scriptPubKey
verifying_tx = utils.sha256_twice(tx["vin"], tx["vout"], scriptPubKey)

MULTI_sigs = []
P2PKH_sigs = []
pubKey_sigs = []
for i in range(MULTI_cnt):
    MULTI_privKeys[i] = MULTI_privKeys[i].sign(verifying_tx).hex()
    MULTI_sigs.append(MULTI_privKeys[i])

N = MULTIsig_number[::2]
M = MULTIsig_number[1::2]
MULTI_sigs_result = []
idx = 0
for i in range(len(M)):
    c = MULTI_sigs[idx:M[i]]
    MULTI_sigs_result.append(random.sample(c, N[i]))
    
for j in range(P2PKH_cnt):
    P2PKH_privKeys[j] = P2PKH_privKeys[j].sign(verifying_tx).hex()
    P2PKH_sigs.append(P2PKH_privKeys[j])

for k in range(pubKey_cnt):
    pubKey_privKeys[k] = pubKey_privKeys[k].sign(verifying_tx).hex()
    pubKey_sigs.append(pubKey_privKeys[k])

print(f"redeem script: {origin}")

print("pubKey_sigs: ")
for z in range(pubKey_cnt):
    print(f"{z}: ")
    print(f"{pubKey_sigs[z]}")
    print()
    
print("MULTI_sigs_result: ")
for x in range(len(MULTI_sigs_result)):
    print(f"{x}: ")
    print(f"{MULTI_sigs_result[x]}")
    print()

print("P2PKH_sigs, P2PKH_pubkey: ")
for y in range(len(P2PKH_pubKeys)):
    print(f"{y}: ")
    print("sig:")
    print(f"{P2PKH_sigs[y]}")
    print("pubKey:")
    print(f"{P2PKH_pubKeys[y]}")

In [ ]:
# 2-3-2
# 마지막 출력문에서 scriptSig에 사용될 sig 선택.
# pubKey_sigs는 인덱스마다 redeem script의 앞에서부터 pubKey에 대응하는 sig
# MULTI_sigs는 인덱스마다 redeem script의 앞에서부터 순서대로 CHECKMULTISIG의 sig list
# P2PKH_sigs, P2PKH_pubKeys는 인덱스마다 redeem script의 앞에서부터 순서대로 <pubKeyHash>에 대응하는 sig, pubKey

# scriptSig = " ".join([pubKey_sigs[index], ...])
# scriptSig = " ".join(MULTI_sigs_result[index])
# scriptSig = " ".join([P2PKH_sigs[index], P2PKH_pubKeys[index], ...])

scriptSig = " ".join([pubKey_sigs[1]])
path = " FALSE"

scriptSig = scriptSig + path + " < " + redeem_script + " > "
print(scriptSig)

In [ ]:
# 3.
# update scriptSig
tx["vin"] = utils.scriptSig_update(tx["vin"], scriptSig)

# update txid
txid = utils.sha256_twice(tx["vin"], tx["vout"])
tx["txid"] = txid.hex()

try:
    with open("backup/transaction_backup.json", "r") as file:
        transactions = json.load(file)
except FileNotFoundError:
    transactions = []

try:
    with open("backup/utxo_backup.json", "r") as file:
        utxoes = json.load(file)
except FileNotFoundError:
    utxoes = []

utxoes.append(UTXO)
transactions.append(tx)

print(json.dumps(tx, indent=4))
print(json.dumps(UTXO, indent=4))

In [96]:
# 4. add to json file
temp_tx = [tx]
temp_utxo = [UTXO]
with open("backup/transaction_backup.json", "w") as file:
    json.dump(transactions, file, indent=4)
with open("backup/utxo_backup.json", "w") as file:
    json.dump(utxoes, file, indent=4)
    
with open("json/transactions.json", "w") as file:
    json.dump(temp_tx, file, indent=4)
with open("json/UTXOes.json", "w") as file:
    json.dump(temp_utxo, file, indent=4)